### Betterway 36. 이터레이터나 제너레이터를 다룰 때는 Itertools를 사용하라

In [1]:
import itertools

본인 코드가 이터레이션 하는 부분이 많다면 `help(itertools)`를 보면 피할 방법을 찾을 수 있다.

우선 그 중에서 책에서 안내 하는 주요 3가지를 살펴보자

In [2]:
help(itertools)

Help on built-in module itertools:

NAME
    itertools - Functional tools for creating and using iterators.

DESCRIPTION
    Infinite iterators:
    count(start=0, step=1) --> start, start+step, start+2*step, ...
    cycle(p) --> p0, p1, ... plast, p0, p1, ...
    repeat(elem [,n]) --> elem, elem, elem, ... endlessly or up to n times
    
    Iterators terminating on the shortest input sequence:
    accumulate(p[, func]) --> p0, p0+p1, p0+p1+p2
    chain(p, q, ...) --> p0, p1, ... plast, q0, q1, ...
    chain.from_iterable([p, q, ...]) --> p0, p1, ... plast, q0, q1, ...
    compress(data, selectors) --> (d[0] if s[0]), (d[1] if s[1]), ...
    dropwhile(pred, seq) --> seq[n], seq[n+1], starting when pred fails
    groupby(iterable[, keyfunc]) --> sub-iterators grouped by value of keyfunc(v)
    filterfalse(pred, seq) --> elements of seq where pred(elem) is False
    islice(seq, [start,] stop [, step]) --> elements from
           seq[start:stop:step]
    starmap(fun, seq) --> fun(*seq[0

#### 1. 여러 이터레이터 연결하기
- chain : 여러 이터레이터를 하나의 순차적인 이터레이터로 합치고 싶을 때
- repeat : 한 값을 계속 반복해 내놓고 싶을 때 
- cycle : 이터레이터가 반환하는 값을 계속 반복하고 싶을 때
- tee : 이터레이터를 병렬적으로 인자의 개수 만큼 생성하고 싶을 때, 이터레이터의 크기가 달라 속도의 차이가   
        발생하게 되면 큐에 저장 하기 때문에 메모리 사용량이 증가된다.
- zip_longest : zip을 사용 하여 여러 이터레이터에 대응 하지만, 이터레이터의 크기가 다르다면 fillvalue로 None 값을 생성 할 수 있기 때문에 서로 크기가 다른 iterator의 값이 손실 되는 것을 막기 위해 사용한다

In [6]:
# chain

it = itertools.chain([1,2,3], [4,5,6])

print(list(it))

[1, 2, 3, 4, 5, 6]


In [9]:
# repeat

it = itertools.repeat("effective python", 5)
print(list(it))

['effective python', 'effective python', 'effective python', 'effective python', 'effective python']


In [54]:
# cycle

it = itertools.cycle([1,2])

result = [next(it) for _ in range(10)]

print(result)

[1, 2, 1, 2, 1, 2, 1, 2, 1, 2]


In [75]:
# itertator cycle을 generator를 대상으로 사용 했었던 yield from을 쓰려면 이렇게 해야 하는건가..?

def gen(iter) :
    for val in range(iter) :
        yield val

print(f"gen function type is {type(gen(1))}")

def cycle() :
    yield from gen(10)

print(f"cycle function type is {type(cycle())}")

def run(func) :
    print("val : ",end = "")
    for delta in func() :
        print(f"{delta}", end = " ")

run(cycle)

gen function type is <class 'generator'>
cycle function type is <class 'generator'>
val : 0 1 2 3 4 5 6 7 8 9 

In [77]:
# tee

it1, it2, it3 = itertools.tee(["하나", "둘"], 3)

print(list(it1))
print(list(it2))
print(list(it3))



['하나', '둘']
['하나', '둘']
['하나', '둘']


In [88]:
# arguments로 값을 선언 하면 어떻게 될까?
it, *another = itertools.tee(["하나", "둘"], 3)

print(list(it))
print(another)
print(list(another[0]))
print(list(another[1]))

['하나', '둘']
[<itertools._tee object at 0x7fdd18b4fec0>, <itertools._tee object at 0x7fdd18b4f240>]
['하나', '둘']
['하나', '둘']


In [89]:
# zip_longest

keys = ["하나", "둘", "셋"]
values = [1,2,]

normal = list(zip(keys, values))
print(f"list & zip = {normal}")

it = itertools.zip_longest(keys, values, fillvalue="None")
longest = list(it)

print(f"zip_longest : {longest}")


list & zip = [('하나', 1), ('둘', 2)]
zip_longest : [('하나', 1), ('둘', 2), ('셋', 'None')]


#### 2. 이터레이터에서 원소 거르기
- islice : 이터레이터에 원소에 대한 슬라이싱을 진행 할 때 
- takehwile : 이터레이터에서 조건이 True가 되는 순서부터 반환
- dropwhile : 이터레이터에서 Not True가 되는 순서부터 반환
- filterfalse : 메서드명 직역 그대로 이터레이터에서 조건에 false에 해당하는 dropwhile과 비슷하지만, filterfalse는 조건에 해당하지 않는 값들만 반환 하고, dropwhile은 false가 되는 순간 부터 값을 반환한다.

- [참고자료](https://velog.io/@ohjiae/itertools)



In [99]:
# islice
values = list(range(1, 11))

first_five = itertools.islice(values, 5)
print(f"앞에서 다섯 개 : {list(first_five)}")

middle_odds = itertools.islice(values, 2, 8, 2)
print(f"중간의 홀 수들 : {list(middle_odds)}")

앞에서 다섯 개 : [1, 2, 3, 4, 5]
중간의 홀 수들 : [3, 5, 7]


In [101]:
values = list(range(1, 11))
condition = lambda x : x < 7
it = itertools.takewhile(condition, values)

print(list(it))

[1, 2, 3, 4, 5, 6]


In [103]:
# dropwhile
values = list(range(1, 11))
condition = lambda x : x < 7
it = itertools.dropwhile(condition, values)

print(list(it))

[7, 8, 9, 10]


In [113]:
# filterfalse

values = list(range(1, 11))
evens = lambda x : x % 2 == 0
filter_result = filter(evens, values)

print(f"filter : {list(filter_result)}")

it = itertools.filterfalse(evens, values)
print(f"fliter false : {list(it)}")

filter : [2, 4, 6, 8, 10]
fliter false : [1, 3, 5, 7, 9]


#### 3. 이터레이터에서 원소의 조합 만들어내기

- accumulate : 이터레이터에서 원소끼리 합계를 계산하고 누적된 값을 리턴하며 한 번에 한 단계씩 결과를 내놓는다.
    - 누적 연산을 수행
    - 삼각수 관련 문제(누적합)를 풀 때 도움 받은 함수 였다고 함.
    - 속도가 훨씬 빠르다고 함
- product : p를 n개의 길이를 가진 순열을 튜플로 출력한다. (중복순열)
- permutations : p를 n개의 길이를 가진 순열을 튜플로 출력한다. (원소 중복 X) -> 자기 자신을 포함 하지 않음
- combinations : 이터레이터의 순차적인 방향으로 n개의 길이를 갖은 조합을 만들어낸다. 중복 허용 X + 앞에 지나간 값은 다시 반복 하지 않음
- combinations_with_replacement : 이터레이터의 순차적인 방향으로 n개의 길이를 갖은 조합을 만들어낸다. 중복 허용






In [121]:
# accumulate

values = list(range(1, 11))

sum_reduce = itertools.accumulate(values)
print(f"합계 {list(sum_reduce)}")

def sum_modulo_20(first, second) :
    output = first + second
    return output % 20

modulo_reduce = itertools.accumulate(values, sum_modulo_20)
print(f"20으로 나눈 나머지의 합계 : {list(modulo_reduce)}")

합계 [1, 3, 6, 10, 15, 21, 28, 36, 45, 55]
20으로 나눈 나머지의 합계 : [1, 3, 6, 10, 15, 1, 8, 16, 5, 15]


In [123]:
# product

single = itertools.product([1,2,], repeat = 2)
print(f"리스트 한 개 : {list(single)}")

multiple = itertools.product([1,2,], ["a","b",])
print(f"리스트 여러 개 : {list(multiple)}")

리스트 한 개 : [(1, 1), (1, 2), (2, 1), (2, 2)]
리스트 여러 개 : [(1, 'a'), (1, 'b'), (2, 'a'), (2, 'b')]


In [127]:
# permutations

it = itertools.permutations([1,2,3,4], 2) # 인자 값은 2개의 순열을 생성, 자기 자신은 포함 하지 않음

print(list(it))

[(1, 2), (1, 3), (1, 4), (2, 1), (2, 3), (2, 4), (3, 1), (3, 2), (3, 4), (4, 1), (4, 2), (4, 3)]


In [129]:
# combinations

it = itertools.combinations([1,2,3,4,], 2) # 조합에서는 중복을 허용 하지 않는다

print(list(it))

[(1, 2), (1, 3), (1, 4), (2, 3), (2, 4), (3, 4)]


In [130]:
# combinations_with_replacement

it = itertools.combinations_with_replacement([1,2,3,4,], 2)

print(list(it))

[(1, 1), (1, 2), (1, 3), (1, 4), (2, 2), (2, 3), (2, 4), (3, 3), (3, 4), (4, 4)]


In [2]:
import os

file_size = os.path.getsize("./betterway36.ipynb")

def convert_size(size_bytes):
    import math
    if size_bytes == 0:
        return "0B"
    size_name = ("B", "KB", "MB", "GB", "TB", "PB", "EB", "ZB", "YB")
    i = int(math.floor(math.log(size_bytes, 1024)))
    p = math.pow(1024, i)
    s = round(size_bytes / p, 2)
    return "%s %s" % (s, size_name[i])

In [3]:
convert_size(file_size)

'38.97 KB'